In [ ]:
pip install requests beautifulsoup4 pandas

In [ ]:
import requests
from bs4 import BeautifulSoup

def scrape_gsmarena(url):
    headers = {
        "User-Agent": "Mozilla/5.0"
    }

    response = requests.get(url, headers=headers)
    soup = BeautifulSoup(response.text, "html.parser")

    # Phone name
    name_element = soup.find("h1", class_="specs-phone-name-title")
    name = name_element.text.strip() if name_element else "N/A"

    specs = {"name": name}

    # Extract all spec categories
    specs_div = soup.find("div", id="specs-list")
    if not specs_div:
        return specs # Return if no specs-list div found

    current_category = "General Specs"
    for child in specs_div.children:
        if child.name == 'h2':
            current_category = child.text.strip()
        elif child.name == 'table':
            for row in child.find_all("tr"):
                key_element = row.find("td", class_="ttl")
                value_element = row.find("td", class_="nfo")

                if key_element and value_element:
                    key = key_element.text.strip()
                    value = value_element.text.strip()
                    # Clean up the category name
                    cleaned_category = current_category.replace(' ', '_').replace('/', '_').replace('.', '').replace('(', '').replace(')', '')
                    specs[f"{cleaned_category}_{key}"] = value

    return specs


#TEST SCRAPER
url = "https://www.gsmarena.com/samsung_galaxy_s23-12024.php"
data = scrape_gsmarena(url)

for k, v in data.items():
    print(k, ":", v)


name : Samsung Galaxy S23 Ultra
General_Specs_Technology : GSM / CDMA / HSPA / EVDO / LTE / 5G
General_Specs_2G bands : GSM 850 / 900 / 1800 / 1900
General_Specs_ : Samsung DeX, Samsung Wireless DeX (desktop experience support)
Ultra Wideband (UWB) support
General_Specs_3G bands : HSDPA 850 / 900 / 1700(AWS) / 1900 / 2100
General_Specs_4G bands : 1, 2, 3, 4, 5, 7, 8, 12, 13, 17, 18, 19, 20, 25, 26, 28, 32, 38, 39, 40, 41, 66 - International
General_Specs_5G bands : 1, 2, 3, 5, 7, 8, 12, 20, 25, 28, 38, 40, 41, 66, 75, 77, 78 SA/NSA/Sub6 - International
General_Specs_Speed : HSPA, LTE (up to 7CA), 5G
General_Specs_Announced : 2023, February 01
General_Specs_Status : Available. Released 2023, February 17
General_Specs_Dimensions : 163.4 x 78.1 x 8.9 mm (6.43 x 3.07 x 0.35 in)
General_Specs_Weight : 234 g (8.25 oz)
General_Specs_Build : Glass front (Gorilla Glass Victus 2), glass back (Gorilla Glass Victus 2), aluminum frame
General_Specs_SIM : · Nano-SIM + eSIM· Nano-SIM + Nano-SIM + eSI

In [ ]:
import json

with open("s23.json", "w") as f:
    json.dump(data, f, indent=4)

In [ ]:
phones = {
    "s23": "https://www.gsmarena.com/samsung_galaxy_s23-12024.php",
    "iphone_15": "https://www.gsmarena.com/apple_iphone_15-12532.php",
    "iqoo_z7": "https://www.gsmarena.com/vivo_iqoo_z7-12165.php"
}

all_data = []

for name, link in phones.items():
    all_data.append(scrape_gsmarena(link))

# Save entire dataset
json.dump(all_data, open("phones.json", "w"), indent=4)


In [ ]:
import requests
from bs4 import BeautifulSoup
import json

def scrape_gadgets360(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36"
    }

    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        print(f"Failed to load page. Status code: {response.status_code}")
        print(f"Response text (first 500 chars): {response.text[:500]}")
        return None

    soup = BeautifulSoup(response.text, "html.parser")
    data = {}

    # ✅ Phone Name
    try:
        data["name"] = soup.find("h1", class_="product-title").text.strip()
    except:
        data["name"] = None

    # ✅ Price in India
    try:
        price_block = soup.find("div", class_="price")
        data["price"] = price_block.text.strip()
    except:
        data["price"] = None

    # ✅ Highlights (Key Specs)
    highlights = []
    try:
        hl_block = soup.find("div", class_="specs-keypoints")
        for li in hl_block.find_all("li"):
            highlights.append(li.text.strip())
    except:
        pass
    data["highlights"] = highlights

    # ✅ Full Specification Table
    specs = {}
    try:
        tables = soup.find_all("table", class_="table-hover")
        for table in tables:
            category = table.find_previous("h2").text.strip()
            specs[category] = {}

            for row in table.find_all("tr"):
                cols = row.find_all("td")
                if len(cols) == 2:
                    key = cols[0].text.strip()
                    value = cols[1].text.strip()
                    specs[category][key] = value
    except:
        pass

    data["specifications"] = specs
    return data


# ✅ TEST SCRAPER
url = "https://gadgets360.com/mobiles/samsung-galaxy-s23-price-in-india-113953"
phone_data = scrape_gadgets360(url)

print(json.dumps(phone_data, indent=4))


Failed to load page. Status code: 403
Response text (first 500 chars): <HTML><HEAD>
<TITLE>Access Denied</TITLE>
</HEAD><BODY>
<H1>Access Denied</H1>
 
You don't have permission to access "http&#58;&#47;&#47;gadgets360&#46;com&#47;mobiles&#47;samsung&#45;galaxy&#45;s23&#45;price&#45;in&#45;india&#45;113953" on this server.<P>
Reference&#32;&#35;18&#46;ce06d217&#46;1762435302&#46;6f3ae40b
<P>https&#58;&#47;&#47;errors&#46;edgesuite&#46;net&#47;18&#46;ce06d217&#46;1762435302&#46;6f3ae40b</P>
</BODY>
</HTML>

null


In [3]:
pip install selenium webdriver-manager bs4

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.7/9.7 MB 65.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 512.0/512.0 kB 32.0 MB/s eta 0:00:00


In [ ]:
import os
import requests
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from bs4 import BeautifulSoup
import time
import json

# Install Google Chrome stable and its dependencies
!apt-get update
!apt-get install -y libvulkan1 # Install missing dependency
!wget -q https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get install -y --fix-missing

# Set the path to the Chrome binary
CHROME_BIN_PATH = '/usr/bin/google-chrome'

def scrape_flipkart_selenium(url):
    options = webdriver.ChromeOptions()
    options.add_argument("--headless")
    options.add_argument("--no-sandbox")
    options.add_argument("--disable-dev-shm-usage")
    options.add_argument("--disable-gpu")
    options.add_argument("--window-size=1920,1080")
    options.add_argument("--disable-blink-features=AutomationControlled")
    options.add_argument("user-agent=Mozilla/5.0")
    options.binary_location = CHROME_BIN_PATH

    # Use ChromeDriverManager to automatically install and manage ChromeDriver
    # It will find a compatible ChromeDriver for the installed Google Chrome stable
    service = Service(ChromeDriverManager().install())

    driver = webdriver.Chrome(
        service=service,
        options=options
    )

    driver.get(url)
    time.sleep(10) # Increased sleep time for page to load to ensure content loads

    soup = BeautifulSoup(driver.page_source, "html.parser")
    driver.quit()

    product_data = {}

    # Product Name - Updated selector based on user provided HTML
    name_element = soup.select_one("span.VU-ZEz") # Using the specific class provided by the user
    product_data["name"] = name_element.text.strip() if name_element else "N/A"

    # Price (current, original, discount)
    current_price_element = soup.select_one(".Nx9bqj.CxhGGd")
    product_data["current_price"] = current_price_element.text.strip() if current_price_element else "N/A"

    # Original price - Using raw string for selector to avoid SyntaxWarning
    original_price_element = soup.select_one(r".yRaY8j.A6\+E6v")
    product_data["original_price"] = original_price_element.text.strip() if original_price_element else "N/A"

    discount_element = soup.select_one(".UkUFwK.WW8yVX")
    product_data["discount"] = discount_element.text.strip() if discount_element else "N/A"

    # Highlights / Key Features
    highlights = []
    # Refined selectors for highlights/key specs lists - targeting common Flipkart patterns
    highlight_sections = soup.select("div._1mXcCf ul li, div._2l0sN7 ul li, div._3FN5M2 ul li, div._2cM9iY ul li")
    for item in highlight_sections:
        highlights.append(item.text.strip())
    product_data["highlights"] = highlights if highlights else "N/A"

    return product_data


url = "https://www.flipkart.com/redmi-13-5g-black-diamond-128-gb/p/itmbf96c9b15ce5e" # Using a working URL example
product_details = scrape_flipkart_selenium(url)
print(json.dumps(product_details, indent=4))

Hit:1 https://dl.google.com/linux/chrome/deb stable InRelease
Hit:2 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:3 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:4 https://cli.github.com/packages stable InRelease
Hit:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:9 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not s

In [ ]:
import requests
from bs4 import BeautifulSoup

def scrape_croma(url):
    headers = {
        "User-Agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64)"
    }

    response = requests.get(url, headers=headers)
    if response.status_code != 200:
        return {"error": "Failed to fetch page"}

    soup = BeautifulSoup(response.text, "html.parser")

    data = {}

    # ✅ Product Name
    title = soup.select_one("h1.pdp__title")
    data["name"] = title.text.strip() if title else None

    # ✅ Price
    price = soup.select_one(".pdp__priceText")
    data["price"] = price.text.strip() if price else None

    # ✅ Old Price + Discount
    old_price = soup.select_one(".pdp__mrpPrice")
    discount = soup.select_one(".pdp__discount")

    data["old_price"] = old_price.text.strip() if old_price else None
    data["discount"] = discount.text.strip() if discount else None

    # ✅ Highlights / Features
    highlights = []
    highlight_items = soup.select(".key-feature-item li")
    for item in highlight_items:
        highlights.append(item.text.strip())

    data["highlights"] = highlights if highlights else None

    # ✅ Availability
    availability = soup.select_one(".pdp__delivery-details")
    data["availability"] = availability.text.strip() if availability else "In stock"

    return data


# ✅ Example usage
url = "https://www.croma.com/samsung-galaxy-s24-5g-256gb/p/275944"  # Replace with any Croma mobile link
result = scrape_croma(url)

print(result)


{'error': 'Failed to fetch page'}


In [15]:
import os
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import json

# Install Google Chrome stable and its dependencies
# These commands should ideally run once per session, but including them in the cell
# ensures they are executed if the runtime resets.
!apt-get update
!apt-get install -y libvulkan1 # Install missing dependency
!wget -q https://dl.google.com/linux/direct/google-chrome-stable_current_amd64.deb
!dpkg -i google-chrome-stable_current_amd64.deb
!apt-get install -y --fix-missing

# Set the path to the Chrome binary
CHROME_BIN_PATH = '/usr/bin/google-chrome'

options = Options()
options.add_argument("--headless=new")
options.add_argument("--no-sandbox")
options.add_argument("--disable-dev-shm-usage")
options.add_argument("--disable-gpu")
options.add_argument("--window-size=1920,1080")
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("user-agent=Mozilla/5.0")
options.binary_location = CHROME_BIN_PATH

# Initialize the WebDriver once outside the loop
service = Service(ChromeDriverManager().install())
driver = webdriver.Chrome(
    service=service,
    options=options
)

all_phones_data = []
base_url = "https://www.flipkart.com/search?q=mobile+phones"
max_pages_to_scrape = 10 # To get at least 200 results (Flipkart typically shows 24 items per page)

print(f"Starting to scrape up to {max_pages_to_scrape} pages from Flipkart...")

for page_num in range(1, max_pages_to_scrape + 1):
    current_url = f"{base_url}&page={page_num}"
    print(f"Scraping page {page_num}: {current_url}")

    driver.get(current_url)
    time.sleep(5) # Increased sleep time for pages to fully load and dynamic content to render

    soup = BeautifulSoup(driver.page_source, "html.parser")

    product_cards = soup.find_all("div", class_="_75nlfW")

    if not product_cards:
        print(f"No product cards found on page {page_num}. Ending pagination.")
        break

    for card in product_cards:
        product_data = {}

        # Product Name
        name_element = card.find("div", class_="KzDlHZ")
        product_data["name"] = name_element.text.strip() if name_element else "N/A"

        # Current Price
        current_price_element = card.find("div", class_="Nx9bqj")
        product_data["current_price"] = current_price_element.text.strip() if current_price_element else "N/A"

        # Original Price
        original_price_element = card.find("div", class_="yRaY8j")
        product_data["original_price"] = original_price_element.text.strip() if original_price_element else "N/A"

        # Discount
        discount_element = card.find("div", class_="UkUFwK")
        product_data["discount"] = discount_element.text.strip() if discount_element else "N/A"

        # Ratings
        rating_element = card.find("div", class_="XQDdHH")
        product_data["rating"] = rating_element.text.strip() if rating_element else "N/A"

        # Number of Ratings and Reviews
        rating_review_element = card.find("span", class_="Wphh3N")
        if rating_review_element:
            text = rating_review_element.text.strip()
            # The text might be like '23,292 Ratings & 1,123 Reviews'
            parts = text.split('&')
            product_data["total_ratings"] = parts[0].replace('Ratings', '').strip() if len(parts) > 0 else "N/A"
            product_data["total_reviews"] = parts[1].replace('Reviews', '').strip() if len(parts) > 1 else "N/A"
        else:
            product_data["total_ratings"] = "N/A"
            product_data["total_reviews"] = "N/A"

        # Key Features (Highlights)
        features = []
        feature_list = card.find("ul", class_="G4BRas")
        if feature_list:
            for li in feature_list.find_all("li"):
                features.append(li.text.strip())
        product_data["features"] = features if features else []

        all_phones_data.append(product_data)

    print(f"Scraped {len(product_cards)} phones from page {page_num}. Total phones collected so far: {len(all_phones_data)}")

# Quit the driver after all pages have been scraped
driver.quit()

# Overwrite the 'phones' variable with the collected data
phones = all_phones_data

print(f"Finished scraping. Total {len(phones)} phones collected.")
# print(json.dumps(phones, indent=4)) # Uncomment to print the full JSON output

Hit:1 https://dl.google.com/linux/chrome/deb stable InRelease
Hit:2 https://cli.github.com/packages stable InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:8 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:9 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:11 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:12 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not s

In [16]:
def chunk_mobile_data(data):
    model = data.get("name", "Unknown Model") # Changed from 'model' to 'name' to match Flipkart data
    brand = data.get("brand", "") # 'brand' is not a direct key in Flipkart data, will likely be empty unless parsed from name

    chunks = []

    def add_chunk(key, value):
        if value and value != "":
            chunks.append(f"{model} — {key}: {value}")

    # ✅ Simple fields (GSMArena-like keys, mostly not directly in Flipkart data)
    # These fields will likely remain empty/None as they are not top-level keys in the Flipkart scraped data
    # They would need to be extracted from the 'features' list if desired.
    simple_fields = {
        "Brand": brand,
        "Current Price": data.get("current_price"),
        "Original Price": data.get("original_price"),
        "Discount": data.get("discount"),
        "Rating": data.get("rating"),
        "Total Ratings": data.get("total_ratings"),
        "Total Reviews": data.get("total_reviews"),
        "Display": data.get("display"), # Not directly available, would need parsing from 'features'
        "Processor": data.get("processor"), # Not directly available, would need parsing from 'features'
        "Battery": data.get("battery"), # Not directly available, would need parsing from 'features'
        "Rear Camera": data.get("rear_camera"), # Not directly available, would need parsing from 'features'
        "Front Camera": data.get("front_camera"), # Not directly available, would need parsing from 'features'
        "Charging": data.get("charging"), # Not directly available, would need parsing from 'features'
        "Dimensions": data.get("dimensions"), # Not directly available, would need parsing from 'features'
        "Weight": data.get("weight"), # Not directly available, would need parsing from 'features'
        "OS": data.get("os"), # Not directly available, would need parsing from 'features'
        "Network Bands": data.get("network_bands"), # Not directly available, would need parsing from 'features'
        "Build": data.get("build"), # Not directly available, would need parsing from 'features'
        "Extras": data.get("extras") # Not directly available, would need parsing from 'features'
    }

    for key, value in simple_fields.items():
        add_chunk(key, value)

    # ✅ Price (using the Flipkart 'current_price' as a general 'price' for consistency if available)
    if "current_price" in data:
        add_chunk("Price", data["current_price"])

    # ✅ Features list (Flipkart)
    features = data.get("features", [])
    for f in features:
        f_clean = f.replace("|", ",").strip()
        chunks.append(f"{model} — Feature: {f_clean}") # Changed key to 'Feature' for clarity

    return chunks

In [17]:
import json

all_phone_chunks = []
for phone_data in phones:
    chunks = chunk_mobile_data(phone_data)
    all_phone_chunks.extend(chunks)

with open("flipkart_phones_chunks.json", "w") as f:
    json.dump(all_phone_chunks, f, indent=4)

In [18]:
import json

with open("flipkart_phones.json", "w") as f:
    json.dump(phones, f, indent=4)